In [30]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [40]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
wd = webdriver.Chrome('chromedriver', options=chrome_options)

In [60]:
wd.get('https://www.instagram.com')

In [61]:
username = wd.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input')
password = wd.find_element_by_css_selector('#loginForm > div > div:nth-child(2) > div > label > input')
login = wd.find_element_by_css_selector('#loginForm > div > div:nth-child(3) > button')

In [62]:
username.send_keys('froma.kim')
password.send_keys('Rladj@9518423')
login.click()

In [67]:
sendcode = wd.find_element_by_css_selector('#react-root > section > div > div > div.GA2q- > form > span > button')
sendcode.click()

In [69]:
security = wd.find_element_by_css_selector('input')
security.click()

code = input('Keys:')

security.send_keys(code)

Keys:271094


In [71]:
submit = wd.find_element_by_css_selector('#react-root > section > div > div > div.GA2q- > form > span > button')
submit.click()

In [73]:
wd.save_screenshot('current.png')

True